In [1]:
from wayne_utils import load_data, save_data
from cypher_parser import split_cypher_clauses, process_cypher
from copy import deepcopy
import re
tests = load_data( "/home/jiangpeiwen2/jiangpeiwen2/IRA/data/SpCQL/test.json", "json")       # 2007
trains = load_data( "/home/jiangpeiwen2/jiangpeiwen2/IRA/data/SpCQL/train.json", "json")     # 7001

## 构建意图识别prompt

In [2]:
Intentions = """你是一位图数据库语言专家，为了实现将用户自然语言查询转换为图数据库查询语言的目标，你需要首先做到对用户查询的意图识别。
所谓意图识别，就是要根据用户的自然语言查询，判断用户希望查询的对象是什么，以及用户希望以怎样的形式返回查询结果。

首先，根据图数据库的设计哲学，我们定义用户查询的对象可以分为以下几种类型：
（1）节点：图中的实体，比如人、物、地点等。节点可以单独存在。
（2）关系：图节点之间的联系，比如人与人之间的关系、物与物之间的关系等。一个关系依赖于两个节点。
（3）路径：图中实体与关系的组合，是连通的多个节点和关系。路径可以是简单的两个节点之间的关系，也可以是复杂的多个节点之间的关系。

其次，明确用户查询意图中的对象类型、对象定义和约束外，还需要明确用户希望以何种形式返回查询结果。主要有以下几种形式：
（1）最常见的情况，是返回满足查询条件的对象的各种属性值，例如返回满足条件的标签为'人'的节点的'name'属性值。
（2）对返回对象进行计数、排序、去重、限制数量等操作，例如返回满足条件的节点的个数、对节点的属性值进行排序、限制返回数量为5等。
请你根据用户输入的自然语言查询，判断用户查询的对象类型和返回形式。并根据上述类定义以下列示例结构化字典形式返回结果：
{   
    '对象':  //对象属性，复杂查询中可能有多个节点、关系或列表
    '约束' : //对上述定义中标签和属性的约束，例如存在性约束、值约束等
    '返回形式': { 
        '总体形式': '对象整体'或'对象属性', 如果有多个对象，则每个对象一个字典
        '聚合操作': '计数'或'求和'或'平均值'或'最大值'或'最小值'，// 没有则填'无',
        '是否去重': '是'或'否',
        '是否排序': '不排序'或'升序'或'降序',
        '是否限制数量'：'否'或具体数量,
        '是否跳过前几个'：'否'或具体数量,
    }
}
下面请你根据用户输入的自然语言查询，判断用户查询的对象类型和返回形式。并根据上述类定义以下列示例结构化字典形式返回结果。
"""


Input = """
用户输入：{INPUT}
意图识别：
"""


Output = """
对象: {intention},
约束: {restrict},
返回形式: {return_s}
"""

In [4]:
def get_return_intention( pasered_dict ):
    ret_intent = {
        '总体形式': None,               # '对象整体'或'对象属性'                       // 必填
        '聚合操作': None,               # '计数'或'求和'或'平均值'或'最大值'或'最小值'，// 没有则填'无'
        '是否去重': None,               # '是'或'否'
        '是否排序': None,               # '不排序'或'desc'或'asc'
        '是否限制数量': None,           # '否'或具体数量
        '是否跳过前几个': None,          # '否'或具体数量,
    }

    # 总体形式
    ret_intent['总体形式'] = '对象整体' if '.' not in pasered_dict['return'] else '对象属性'
    # 属性值列表
    '''
    properties = []
    for pro in [".name", ".location", ".time"]:
        if pro in pasered_dict['return']:
            properties.append( pro[1:] )
    ret_intent['属性值列表'] = properties'''
    # 聚合操作
    aggs = []
    for agg in ['count', 'sum', 'avg', 'max', 'min']:
        if agg in pasered_dict['return']:
            aggs.append( agg )
    ret_intent['聚合操作'] = aggs
    # 是否去重
    ret_intent['是否去重'] = '否' if 'distinct' not in pasered_dict['return'] else '是'
    # 是否排序
    if 'order by' not in pasered_dict:
        ret_intent['是否排序'] = '不排序'  
    else:
        ret_intent['是否排序'] = '降序' if 'desc' in pasered_dict['order by'] else '升序'
    # 是否限制数量
    ret_intent['是否限制数量'] = '否' if 'limit' not in pasered_dict else pasered_dict['limit']
    # 是否限制数量
    ret_intent['是否跳过前几个'] = '否' if 'skip' not in pasered_dict else pasered_dict['skip']
    return ret_intent
def get_restrict( cypher ):
    clause_dict = split_cypher_clauses( cypher.lower() )
    if "where" in clause_dict:
        return clause_dict["where"]
    else:
        return "无约束"

In [5]:
train_intension_list = load_data( "trains_intentions.json", "json")
test_intension_list = load_data( "tests_intentions.json", "json")
def get_prompt( lists, intension_list, train=True):
    ret_list = []
    for i in range( len(lists)):
        nl = lists[i]['query'].lower()
        cypher = lists[i]['cypher'].lower()
        intention = intension_list[i]
        restrict = get_restrict( cypher )
        clause_dict = split_cypher_clauses( cypher )
        clause_dict_new, variables  = process_cypher( clause_dict )
        return_s = get_return_intention( clause_dict_new )
        if train:
            prompt = {
                "instruction": Intentions,
                "input" : Input.format( INPUT = nl),
                "output": Output.format( intention = intention, restrict = restrict, return_s = return_s)
            }
        else:
            prompt = Intentions + Input.format( INPUT = nl)
        ret_list.append( prompt )
    return ret_list
ft_list = get_prompt( trains, train_intension_list, train=True)
save_data( ft_list, "MyMethod_SpCQL_ft.json")

prompt_list = get_prompt( tests, test_intension_list, train=False)
save_data( prompt_list, "MyMethod_SpCQL_intention_prompt_list.pickle")
    

In [6]:
def get_labels( lists, intension_list ):
    ret_list = []
    for i in range( len(lists)):
        cypher = lists[i]['cypher'].lower()
        intention = intension_list[i]
        restrict = get_restrict( cypher )
        clause_dict = split_cypher_clauses( cypher )
        clause_dict_new, variables  = process_cypher( clause_dict )
        return_s = get_return_intention( clause_dict_new )
        # Output.format( intention = intention, restrict = restrict, return_s = return_s)
        ret_list.append( {
            "对象": intention,
            "约束": restrict,
            "返回形式": return_s
        } )
    return ret_list

In [ ]:
prompt_labels = get_labels( tests, test_intension_list )
ft_intention = get_labels( trains, train_intension_list )
save_data( ft_intention, "MyMethod_SpCQL_intention_trains_list.json")
save_data( prompt_labels, "MyMethod_SpCQL_intention_labels_list.json")

## 从Cypher抽取意图

In [1]:
from SpCQL_intention_data import divide_cypher_type, get_left_right, get_two_orient, extract_names_single, extract_names_and_commas
from SpCQL_intention_data import get_no_orient, get_single_node, get_union_node, get_other_node, get_path_node, get_not_comma

In [5]:
def batch_intention_cypher( cypher_list, test):
    paths, single_left, single_right, two_orient, no_orient, single_node, multi_match, others, not_comma_others = divide_cypher_type( cypher_list )
    left_intentions_list = get_left_right( single_left, left=True)
    right_intentions_list = get_left_right( single_right, left=False)       # test 754  train 2578
    two_orient_list = get_two_orient( two_orient, test=test)
    no_orient_list = get_no_orient( no_orient )
    single_node_list = get_single_node( single_node )
    union_node_list = get_union_node( multi_match )
    other_node_list = get_other_node( others )
    path_list = get_path_node( paths )
    if not test:
        not_comma_list = get_not_comma( not_comma_others )
    else:
        not_comma_list = []

    # 后处理
    test_intentions_dict = {}
    for lists in [left_intentions_list, right_intentions_list, two_orient_list, no_orient_list, single_node_list,
                union_node_list, other_node_list, path_list, not_comma_list]:
        for i in range( len(lists)):
            index = lists[i][0]
            intens = lists[i][1]
            test_intentions_dict[ index ] = intens
    test_intentions_dlisty = [ test_intentions_dict[i] for i in range(len(test_intentions_dict))]
    return test_intentions_dlisty

test_intentions_dlisty = batch_intention_cypher( tests, test=True)

In [6]:
save_data( test_intentions_dlisty, "tests_intentions.json")